# Battery consumption analysis

This notebook contains the code used to compute the battery consumption of the developed system on its two configurations:

- C1: smartwatch (sensing) + smartphone (computing)
- C2: smartphone (sensing + computing)

**Input:**

- `04_BATTERY/00_battery-historian.csv`: CSV file containing the battery consumption estimates (%) reported by the _bug_reports_ (`04_BATTERY/bug-reports/`) analysed using the BatteryHistorian tool. Each battery consumption estimate is associated to a device and a certain number of TUG executions.

**Output:**

- `04_BATTERY/01_battery-consumption.csv` (**Table 7**): Consumption estimates (% and mAh) for TUG execution in each device.

In [1]:
import os
import pandas as pd

from IPython.display import Markdown as md

## Constants

In [2]:
BATTERY_PATH = '04_BATTERY'
BATTERY_FILE = os.path.join(BATTERY_PATH, '00_battery-historian.csv')
BATTERY_RESULTS = os.path.join(BATTERY_PATH, '01_battery-consumption.csv')

SP_CAPACITY_MAH = 5160
SW_CAPACITY_MAH = 580

## Functions

In [3]:
def mA_consumption_from_percentage(x, percentage_attr):
    percentage = x[percentage_attr]
    return percentage * SP_CAPACITY_MAH / 100 if x.name.startswith('sp') else percentage * SW_CAPACITY_MAH / 100

## Analysis

First, we load the data from the `04_BATTERY/battery-historian.csv` file, where each row contains information regarding battery consumption (i.e., consumption records), and each row has the following attributes:

- `executions`: number of TUG executions performed in this consumption record.
- `device`: from which device (_sp_, _sp-paired_, or _sw_) is the consumption record.
- `consumption`: percentaje of consumed battery.

This information was manually extracted from the bugreports in `04_BATTERY/bug-reports/` using the [BatteryHistorian](https://github.com/google/battery-historian) tool.

In [4]:
battery_df = pd.read_csv(BATTERY_FILE)
battery_df

,executions,device,consumption
0,60,sp,0.33
1,60,sp,0.36
2,40,sp,0.26
3,10,sp,0.06
4,10,sp,0.07
5,10,sp,0.08
6,10,sp,0.06
7,10,sp,0.05
8,10,sp,0.05
9,10,sp,0.06


Then, we compute the battery percentaje consumption per TUG execution (i.e., ratio) for each consumption record.

In [5]:
battery_df['ratio'] = battery_df['consumption'] / battery_df['executions']
battery_df

,executions,device,consumption,ratio
0,60,sp,0.33,0.005500
1,60,sp,0.36,0.006000
2,40,sp,0.26,0.006500
3,10,sp,0.06,0.006000
4,10,sp,0.07,0.007000
5,10,sp,0.08,0.008000
6,10,sp,0.06,0.006000
7,10,sp,0.05,0.005000
8,10,sp,0.05,0.005000
9,10,sp,0.06,0.006000


Next, we group the ratios per devices to obtain the mean consumption per device and TUG execution

In [6]:
battery_by_device_df = battery_df[['device', 'ratio']].groupby(['device']).mean()
battery_by_device_df

,ratio
device,
sp,0.005958
sp-paired,0.004920
sw,0.009966


Finally, we compute mAh corresponding to those mean consumptions taking into account that:

- The smartphones (_sp_ and _sp-paired_) have a 5160 mAh battery.
- The smartwatch (_sw_) has a 580 mAh battery.

In [7]:
battery_by_device_df['ratio_ma'] = battery_by_device_df.apply(lambda x: mA_consumption_from_percentage(x, 'ratio'), axis=1)
battery_by_device_df

,ratio,ratio_ma
device,,
sp,0.005958,0.307450
sp-paired,0.004920,0.253854
sw,0.009966,0.057801


In [8]:
sp_consumption, sp_ratio = battery_by_device_df.loc['sp']['ratio_ma'], battery_by_device_df.loc['sp']['ratio']
sw_consumption, sw_ratio = battery_by_device_df.loc['sw']['ratio_ma'], battery_by_device_df.loc['sw']['ratio']
sp_paired_consumption, sp_paired_ratio = battery_by_device_df.loc['sp-paired']['ratio_ma'], battery_by_device_df.loc['sp-paired']['ratio']


md(f'''
**TUG System consumption**

**C1 (smartwach + smartphone)**

- Smartwatch consumption: {sw_consumption:.4f} ({sw_ratio:.4f} %)
- Smartphone consumption: {sp_paired_consumption:.4f} ({sp_paired_ratio:.4f} %)
- Combined consumption: {(sw_consumption + sp_paired_consumption):.4f}

**C2 (smartphone)**

- Consumption: {sp_consumption:.4f} ({sp_ratio:.4f} %)

**This information is reported in Table 7 of the paper.**
''')


**TUG System consumption**

**C1 (smartwach + smartphone)**

- Smartwatch consumption: 0.0578 (0.0100 %)
- Smartphone consumption: 0.2539 (0.0049 %)
- Combined consumption: 0.3117

**C2 (smartphone)**

- Consumption: 0.3075 (0.0060 %)

**This information is reported in Table 6 of the paper.**


In [9]:
battery_by_device_df.to_csv(BATTERY_RESULTS)